In [20]:
import pandas as pd

# Đường dẫn đến tệp Excel
file_path = './input/label.xlsx'

# Đọc tệp Excel
datatimeOA = pd.read_excel(file_path)

# Hiển thị các dòng đầu tiên của DataFrame
print(datatimeOA.head())
datatimeOA['StartDate'] = pd.to_datetime(datatimeOA['StartDate'], format='%H:%M:%S')

# Tính toán endDate bằng cách thêm duration (tính bằng giờ) vào StartDate
datatimeOA['endDate'] = datatimeOA['StartDate'] + pd.to_timedelta(datatimeOA['duration'], unit='s')

# Hiển thị các dòng đầu tiên của DataFrame sau khi tính toán
print(datatimeOA.head())


  StartDate  duration  hy
0  12:43:06     18.49  hy
1  12:44:55     13.36  hy
2  12:46:27     10.55  hy
3  12:46:41     10.60  hy
4  12:46:56     10.80  hy
            StartDate  duration  hy                 endDate
0 1900-01-01 12:43:06     18.49  hy 1900-01-01 12:43:24.490
1 1900-01-01 12:44:55     13.36  hy 1900-01-01 12:45:08.360
2 1900-01-01 12:46:27     10.55  hy 1900-01-01 12:46:37.550
3 1900-01-01 12:46:41     10.60  hy 1900-01-01 12:46:51.600
4 1900-01-01 12:46:56     10.80  hy 1900-01-01 12:47:06.800


In [21]:
datatimeOA['endDate'] = pd.to_datetime(datatimeOA['endDate'], format='%H:%M:%S:%f')

In [22]:
import pandas as pd

# Đường dẫn đến tệp Excel
file_path = './input/flowcmh20.xlsx'

# Đọc tệp Excel
flowData = pd.read_excel(file_path)



# Hiển thị các dòng đầu tiên của DataFrame
print(flowData.head())

           time     value
0  00:37:05:162  0.148586
1  00:37:05:167  0.148548
2  00:37:05:172  0.148485
3  00:37:05:177  0.148397
4  00:37:05:182  0.148284


In [23]:
def convert_hour_to_12(dt):
    if dt.hour == 0:
        return dt.replace(hour=12)
    return dt

In [24]:
flowData['time'] = pd.to_datetime(flowData['time'], format='%H:%M:%S:%f')
flowData['time'] = flowData['time'].apply(convert_hour_to_12)
flowData['nhan'] = 0
print(flowData.head())

                     time     value  nhan
0 1900-01-01 12:37:05.162  0.148586     0
1 1900-01-01 12:37:05.167  0.148548     0
2 1900-01-01 12:37:05.172  0.148485     0
3 1900-01-01 12:37:05.177  0.148397     0
4 1900-01-01 12:37:05.182  0.148284     0


In [25]:
for index, row in datatimeOA.iterrows():
    mask = (flowData['time'] >= row['StartDate']) & (flowData['time'] <= row['endDate'])
    flowData.loc[mask, 'nhan'] = 1
    

print(flowData.head())

                     time     value  nhan
0 1900-01-01 12:37:05.162  0.148586     0
1 1900-01-01 12:37:05.167  0.148548     0
2 1900-01-01 12:37:05.172  0.148485     0
3 1900-01-01 12:37:05.177  0.148397     0
4 1900-01-01 12:37:05.182  0.148284     0


In [26]:
flowData_nhan_1 = flowData[flowData['nhan'] == 1]
print(flowData_nhan_1)

                           time     value  nhan
72168   1900-01-01 12:43:06.002 -0.065932     1
72169   1900-01-01 12:43:06.007 -0.065745     1
72170   1900-01-01 12:43:06.012 -0.065540     1
72171   1900-01-01 12:43:06.017 -0.065319     1
72172   1900-01-01 12:43:06.022 -0.065085     1
...                         ...       ...   ...
1048570 1900-01-01 02:04:28.012  0.196029     1
1048571 1900-01-01 02:04:28.017  0.203181     1
1048572 1900-01-01 02:04:28.022  0.210161     1
1048573 1900-01-01 02:04:28.027  0.216970     1
1048574 1900-01-01 02:04:28.032  0.223612     1

[598923 rows x 3 columns]


In [27]:
new_file_path = './output/flowData_modified.xlsx'

# Lưu DataFrame vào tệp Excel mới
flowData.to_excel(new_file_path, index=False)

In [28]:
import math
def create_training_data(data, window_size, step_size):

    _train = []
    _label = []

    for i in range(0, data.shape[0] - window_size, step_size):
        try:
            xs = data['value'].values[i: i + window_size]
        
            # Skip examples where the activity label changes within the window.
            if (data['nhan'][i+1] != data['nhan'][i + window_size-1]):
                print(f"Skipping index {i} due to label change within window.")
                continue

            label = data['nhan'][i + window_size-1]

            # Skip examples where the label is NaN.
            if math.isnan(label):
                print(f"Skipping index {i} due to NaN label.")
                continue

            _train.append(xs)
            _label.append(label)
        
        except Exception as e:
            print(f"An error occurred at index {i}: {e}")

    return _train, _label

In [29]:
data = pd.DataFrame({
    'value': [1, 2,3, 4, 5, 6],
    'nhan': [0, 0, 0, 1, 1, 1]
})

# Gọi hàm với window_size và step_size
window_size = 30
step_size = 1
_train, _test = create_training_data(data, window_size, step_size)

print("Training data:", _train)
print("Training labels:", _test)

Training data: []
Training labels: []


In [37]:

_train, _label = create_training_data(
    data=flowData, window_size=40, step_size=10)


Skipping index 72130 due to label change within window.
Skipping index 72140 due to label change within window.
Skipping index 72150 due to label change within window.
Skipping index 72160 due to label change within window.
Skipping index 75830 due to label change within window.
Skipping index 75840 due to label change within window.
Skipping index 75850 due to label change within window.
Skipping index 75860 due to label change within window.
Skipping index 93930 due to label change within window.
Skipping index 93940 due to label change within window.
Skipping index 93950 due to label change within window.
Skipping index 93960 due to label change within window.
Skipping index 96610 due to label change within window.
Skipping index 96620 due to label change within window.
Skipping index 96630 due to label change within window.
Skipping index 112330 due to label change within window.
Skipping index 112340 due to label change within window.
Skipping index 112350 due to label change with

In [38]:
print(_train[0])
print(len(_train))
print(len(_label))



[0.14858611 0.14854803 0.14848476 0.14839655 0.14828376 0.14814686
 0.14798641 0.14780309 0.14759765 0.14737093 0.14712388 0.14685753
 0.14657298 0.14627142 0.14595407 0.14562224 0.14527728 0.14492059
 0.14455364 0.14417789 0.14379484 0.14340598 0.14301285 0.14261692
 0.14221969 0.14182262 0.14142712 0.14103456 0.14064625 0.14026348
 0.13988738 0.13951903 0.13915943 0.13880947 0.13846991 0.13814141
 0.13782451 0.13751966 0.13722711 0.13694706]
103498
103498


In [39]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(_train, _label, test_size=0.2, random_state=42)

In [40]:
import numpy as np


x_train = np.array(x_train)
x_test = np.array(x_test)
y_train = np.array(y_train)
y_test = np.array(y_test)


In [41]:
print(x_train.shape)
print(y_train)

(82798, 40)
[0 1 1 ... 0 0 1]


In [46]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense

# 3. Xây dựng mô hình CNN
model = Sequential()
model.add(Conv1D(filters=32, kernel_size=3, activation='relu', input_shape=(40, 1)))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(100, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

# 4. Biên dịch mô hình
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# 5. Huấn luyện mô hình
model.fit(x_train, y_train, epochs=10, batch_size=32, validation_split=0.2)


Epoch 1/50
2070/2070 [==============================] - 3s 1ms/step - loss: 0.4539 - accuracy: 0.8089 - val_loss: 0.4456 - val_accuracy: 0.8121
Epoch 2/50
2070/2070 [==============================] - 3s 1ms/step - loss: 0.4353 - accuracy: 0.8194 - val_loss: 0.4408 - val_accuracy: 0.8169
Epoch 3/50
2070/2070 [==============================] - 3s 1ms/step - loss: 0.4334 - accuracy: 0.8209 - val_loss: 0.4355 - val_accuracy: 0.8152
Epoch 4/50
2070/2070 [==============================] - 3s 1ms/step - loss: 0.4318 - accuracy: 0.8224 - val_loss: 0.4385 - val_accuracy: 0.8163
Epoch 5/50
2070/2070 [==============================] - 3s 2ms/step - loss: 0.4308 - accuracy: 0.8224 - val_loss: 0.4308 - val_accuracy: 0.8176
Epoch 6/50
2070/2070 [==============================] - 3s 2ms/step - loss: 0.4292 - accuracy: 0.8220 - val_loss: 0.4320 - val_accuracy: 0.8201
Epoch 7/50
2070/2070 [==============================] - 3s 1ms/step - loss: 0.4279 - accuracy: 0.8229 - val_loss: 0.4300 - val_accuracy:

In [36]:
# 6. Đánh giá mô hình
loss, accuracy = model.evaluate(x_train, y_train)
print(f'Loss: {loss}, Accuracy: {accuracy}')

2605/2605 [==============================] - 2s 614us/step - loss: 0.4343 - accuracy: 0.8197
Loss: 0.43430688977241516, Accuracy: 0.8196760416030884
